In [7]:
import pandas as pd
import urllib.parse
import sqlalchemy
from sqlalchemy import create_engine, text
import openpyxl
import re
from datetime import datetime
import pytz
import os
from openpyxl.utils import get_column_letter
from lib.transform import WIPARAging, CreateTableInSQLServer, WIPARRecon, StaffMonthly, StaffList, StaffPosted, ARBalanceListing

class StaffPosted:
    
    def __init__(self, folder_path, utcFormat):
        self.folder_path = folder_path
        self.utcFormat = utcFormat
    def substring_after_5th_whitespace(self,txt):
        parts = txt.split(' ', 3) 
        if len(parts) > 3:
            return parts[3] 
        return '' 
    def process_files(self):
        # Define timezone and current time based on UTC
        utc_minus = pytz.timezone(self.utcFormat)
        current_time_utc_minus = datetime.now(utc_minus).strftime("%Y-%m-%d %H:%M:%S")

        # List to store DataFrames
        df_list = []

        # Get list of .xlsx files
        xlsx_files = [
            f
            for f in os.listdir(self.folder_path)
            if f.endswith(".xlsx") and not (f.startswith("~"))
        ]

        # Process each file
        for file in xlsx_files:
            file_path = os.path.join(self.folder_path, file)
            wb = openpyxl.load_workbook(file_path)
            D = {}
            D_mapping_cols = {
                "PostedHours": ["PostedHours", ""],
                "BankedHoursUsed": ["BankedUsedHours", ""],
                "BusinessHours": ["BusinessHours", ""],
                "VarianceHours": ["VarianceHours", ""],
                "NonbillHours": ["NonbillHours", ""],
                "BillHours": ["Hours", ""],
            }
            sheet = wb.worksheets[1]
            last_row = sheet.max_row
            # Find last row with "Grand Totals:"
            for row in range(last_row - 6, last_row + 1):
                cell = sheet[f"A{row}"]
                if cell.value == "Grand Totals:":
                    last_row = row - 1
                    break

            # Find first row with "Staff ID"
            for row in range(1, last_row):
                if sheet[f"A{row}"].value is not None:
                    if sheet[f"A{row}"].value[:8] == "Staff ID":
                        first_row = row
                        break
            # Mapping cols name for each KPIS

            for row in range(first_row - 2, first_row + 1):
                for col in range(1, sheet.max_column + 1):
                    cell_value = sheet.cell(row=row, column=col).value
                    for col_name in D_mapping_cols:
                        if cell_value and D_mapping_cols[col_name][0] == re.sub(
                            r"[^a-zA-Z]", "", str(cell_value)
                        ):
                            D_mapping_cols[col_name][1] = get_column_letter(col)

            # Find second row with "Staff ID"
            for row in range(first_row + 3, last_row):
                if sheet[f"A{row}"].value is not None:
                    if sheet[f"A{row}"].value[:8] == "Staff ID":
                        second_row = row
                        break
            k_row = second_row - first_row
            # Find first row with value starting with "For Accounting period dates:" to determine begin date and end date
            for row in range(1, 20):
                if sheet[f"I{row}"].value is not None:
                    if sheet[f"I{row}"].value[:28] == "For Accounting period dates:":
                        txt = sheet[f"I{row}"].value
                        match = re.search(
                            r"For Transaction dates:(\d{1,2}/\d{1,2}/\d{4}) - (\d{1,2}/\d{1,2}/\d{4})",
                            txt,
                        )
                        if match:
                            begin_date = match.group(1)
                            end_date = match.group(2)
                            print(f"Begin date: {begin_date}")
                            print(f"End date: {end_date}")
                        else:
                            print("Transaction dates not found.")
                        break

            # Process the data
            for i in range(1, int((last_row - first_row + 1) / k_row + 1)):
                StaffID = self.substring_after_5th_whitespace(sheet[f"A{(i-1) * k_row + first_row}"].value)

                PostedHours = sheet[
                    f"{D_mapping_cols['PostedHours'][1]}{(i-1) * k_row + first_row + 2}"
                ].value
                arr_BankedUsedHrs = sheet[
                    f"{D_mapping_cols['BankedHoursUsed'][1]}{(i-1) * k_row + first_row + 2}"
                ].value.split(" ")
                if len(arr_BankedUsedHrs) == 2:
                    TypeBankedUsed = str(arr_BankedUsedHrs[0])
                    BankedHoursUsed = arr_BankedUsedHrs[1]
                else:
                    TypeBankedUsed = ""
                    BankedHoursUsed = arr_BankedUsedHrs[0]
                BusinessHours = sheet[
                    f"{D_mapping_cols['BusinessHours'][1]}{(i-1) * k_row + first_row + 2}"
                ].value
                VarianceHours = sheet[
                    f"{D_mapping_cols['VarianceHours'][1]}{(i-1) * k_row + first_row + 2}"
                ].value
                NonbillHours = sheet[
                    f"{D_mapping_cols['NonbillHours'][1]}{(i-1) * k_row + first_row + 2}"
                ].value
                BillHours = sheet[
                    f"{D_mapping_cols['BillHours'][1]}{(i-1) * k_row + first_row + 2}"
                ].value
                D[i] = [
                    StaffID,
                    PostedHours,
                    TypeBankedUsed,
                    BankedHoursUsed,
                    BusinessHours,
                    VarianceHours,
                    NonbillHours,
                    BillHours,
                    begin_date,
                    end_date,
                ]

            # Create DataFrame
            df = pd.DataFrame.from_dict(
                D,
                orient="index",
                columns=[
                    "StaffID",
                    "PostedHours",
                    "TypeBankedUsed",
                    "BankedHoursUsed",
                    "BusinessHours",
                    "VarianceHours",
                    "NonbillHours",
                    "BillHours",
                    "begin_date",
                    "end_date",
                ],
            )
            df.fillna(0.0, inplace=True)
            df["RunningTime"] = current_time_utc_minus

            df_list.append(df)
        
        final_df = pd.concat(df_list, ignore_index=True)
        for i in ["RunningTime", "begin_date", "end_date"]:
            final_df[i] = pd.to_datetime(final_df[i])
        return final_df
SQL_SERVER = 'TOM'
DB_Name = 'SSRPA_310492'
User_Name = 'TOM'
Password = 'Silversea'
# Pull all StaffPosted report to table name StaffPosted
result_df = StaffPosted(folder_path=r"C:\Users\tomda\OneDrive\Desktop\Python\Source Spiegel", utcFormat='Etc/GMT-6').process_files()
CreateTableInSQLServer(SQLServerName=SQL_SERVER,DBName=DB_Name, TableName='StaffPosted_Full', UserName=User_Name, PWD=Password, df_data=result_df).run()

Begin date: 1/1/1900
End date: 9/15/2024
OK


In [5]:


s = 'abc asdf asdf 12312asdfasdf'
result = substring_after_2nd_whitespace(s)
print(result)  # Output: 'asdf 12312asdfasdf'
